# Leveraging the `kglab` abstraction layer

Let's try this again -- building a KG -- but this time using the `kglab` library to make things a wee bit easier...

In [1]:
import kglab

namespaces = {
    "wtm": "http://purl.org/heals/food/",
    "ind": "http://purl.org/heals/ingredient/",
    }

kg = kglab.KnowledgeGraph(
    name = "A recipe KG example based on Food.com",
    base_uri = "https://www.food.com/recipe/",
    language = "en",
    namespaces = namespaces,
    )

In [2]:
import rdflib as rdf
from rdflib.namespace import RDF, XSD

In [3]:
node = rdf.URIRef("https://www.food.com/recipe/327593")

kg.add(node, RDF.type, kg.get_ns("wtm").Recipe)
kg.add(node, kg.get_ns("wtm").hasCookTime, rdf.Literal("8", datatype=XSD.integer))
kg.add(node, kg.get_ns("wtm").hasIngredient, kg.get_ns("ind").ChickenEgg)
kg.add(node, kg.get_ns("wtm").hasIngredient, kg.get_ns("ind").CowMilk)
kg.add(node, kg.get_ns("wtm").hasIngredient, kg.get_ns("ind").WholeWheatFlour)

In [4]:
for s, p, o in kg._g:
    print(s, p, o)

https://www.food.com/recipe/327593 http://purl.org/heals/food/hasIngredient http://purl.org/heals/ingredient/ChickenEgg
https://www.food.com/recipe/327593 http://purl.org/heals/food/hasCookTime 8
https://www.food.com/recipe/327593 http://purl.org/heals/food/hasIngredient http://purl.org/heals/ingredient/CowMilk
https://www.food.com/recipe/327593 http://purl.org/heals/food/hasIngredient http://purl.org/heals/ingredient/WholeWheatFlour
https://www.food.com/recipe/327593 http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://purl.org/heals/food/Recipe


In [5]:
s = kg._g.serialize(format="n3")
print(s.decode("utf8"))

@prefix dc: <https://purl.org/dc/terms/> .
@prefix dct: <https://purl.org/dc/dcmitype/> .
@prefix ind: <http://purl.org/heals/ingredient/> .
@prefix owl: <https://www.w3.org/2002/07/owl#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix skos: <https://www.w3.org/2004/02/skos/core#> .
@prefix wtm: <http://purl.org/heals/food/> .
@prefix xml: <http://www.w3.org/XML/1998/namespace> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<https://www.food.com/recipe/327593> a wtm:Recipe ;
    wtm:hasCookTime 8 ;
    wtm:hasIngredient ind:ChickenEgg,
        ind:CowMilk,
        ind:WholeWheatFlour .




Then the serialization becomes much simpler:

In [6]:
kg.save_ttl("tmp.ttl")

In [7]:
kg.save_jsonld("tmp.jsonld")

Next, we'll use the [Parquet](https://parquet.apache.org/) format for *columnar storage*.
This has been especially effective for Big Data frameworks handling data management and analytics effeciently.

In [8]:
kg.save_parquet("tmp.parquet")

In [9]:
import pandas as pd
import os

file_paths = ["tmp.jsonld", "tmp.ttl", "tmp.parquet"]
file_sizes = [os.path.getsize(file_path) for file_path in file_paths]

df = pd.DataFrame({"file_path": file_paths, "file_size": file_sizes})
df

,file_path,file_size
0,tmp.jsonld,779
1,tmp.ttl,680
2,tmp.parquet,3631


Parquet uses compression based on a "dictionary" approach, so it added overhead for small files such as this KG.
We'll revisit this comparison across file formats again with a larger KG.